In [ ]:
import pandas as pd

In [ ]:
3!pip install kaggle


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"haritshandilya","key":"a3b3fc3619ea2aa21d8a587560e99b05"}'}

In [ ]:
#before importing the dataset we want to use this code
# The Kaggle API client expects this file to be in ~/.kaggle,
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d columbine/imdb-dataset-sentiment-analysis-in-csv-format

 35% 9.00M/25.7M [00:00<00:00, 32.3MB/s]
100% 25.7M/25.7M [00:00<00:00, 65.0MB/s]


In [ ]:
from zipfile import ZipFile
file_name="imdb-dataset-sentiment-analysis-in-csv-format.zip"

with ZipFile (file_name,"r") as zip:
  zip.extractall()
  print("Done!")


Done!


In [ ]:
df=pd.read_csv("/content/Train.csv")
df.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [ ]:
###Drop Nan Values

df=df.dropna()


In [ ]:
df.label.value_counts()

0    20019
1    19981
Name: label, dtype: int64

In [ ]:
## Get the Independent Features & Dependent features

X=df.drop('label',axis=1)
y=df['label']

In [ ]:
print(f"X shape = {X.shape}\nY shape = {y.shape}")

X shape = (40000, 1)
Y shape = (40000,)


In [ ]:
import tensorflow as tf

tf.__version__


'2.2.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
### Vocabulary size
voc_size=10000


In [ ]:
statements = X.copy()
statements['text'][0]


'I grew up (b. 1965) watching and loving the Thunderbirds. All my mates at school watched. We played "Thunderbirds" before school, during lunch and after school. We all wanted to be Virgil or Scott. No one wanted to be Alan. Counting down from 5 became an art form. I took my children to see the movie hoping they would get a glimpse of what I loved as a child. How bitterly disappointing. The only high point was the snappy theme tune. Not that it could compare with the original score of the Thunderbirds. Thankfully early Saturday mornings one television channel still plays reruns of the series Gerry Anderson and his wife created. Jonatha Frakes should hand in his directors chair, his version was completely hopeless. A waste of film. Utter rubbish. A CGI remake may be acceptable but replacing marionettes with Homo sapiens subsp. sapiens was a huge error of judgment.'

In [ ]:
statements.reset_index(inplace=True)


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from tqdm import tqdm
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in tqdm(range(0, len(statements))):
    review = re.sub('[^a-zA-Z]', ' ', statements['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
print("Done!")

100%|██████████| 40000/40000 [15:50<00:00, 42.07it/s]

Done!


In [ ]:
corpus[0]  

'grew b watch love thunderbird mate school watch play thunderbird school lunch school want virgil scott one want alan count becam art form took children see movi hope would get glimps love child bitterli disappoint high point snappi theme tune could compar origin score thunderbird thank earli saturday morn one televis channel still play rerun seri gerri anderson wife creat jonatha frake hand director chair version complet hopeless wast film utter rubbish cgi remak may accept replac marionett homo sapien subsp sapien huge error judgment'

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[0]
len(onehot_repr[0])

85

In [ ]:
sent_length=100
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4912 5005 3067]
 [ 463 7452 9951 ... 9687  527 6336]
 [   0    0    0 ... 5223   88 3119]
 ...
 [8047 5652 1314 ... 3322  823 2364]
 [   0    0    0 ... 3798 8591 8591]
 [   0    0    0 ... 5371 7588 8031]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 8006, 1761,  132, 6917, 8531, 4672, 2377,
        132, 4466, 8531, 2377, 1027, 2377, 7112, 9396, 2273, 6399, 7112,
       9796, 3955, 4684, 9831, 8439,  839, 1240, 1606, 1251, 9122, 8097,
       2692, 9778, 6917, 7778, 2568, 6473, 9924, 7126, 1509, 8636, 3073,
       8976, 1036, 6501, 4346, 8531, 5874, 7175, 4467, 6386, 6399, 3067,
       9687, 8320, 4466, 7434, 8547, 5552, 8168, 9311, 5310, 7569, 8475,
       2992, 3152, 1072, 4953, 7700, 4881, 3596,   88, 8424,  908, 1112,
       6892, 9206,   45, 6785, 5794, 1939, 8243, 4523, 8243, 4912, 5005,
       3067], dtype=int32)

In [ ]:
## Creating model
embedding_vector_features=200
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(LSTM(100))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 200)          2000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 2,120,501
Trainable params: 2,120,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape


(40000, (40000,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

X_final.shape,y_final.shape


((40000, 100), (40000,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=104)

In [ ]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=128)

Epoch 1/10
250/250 [==============================] - 9s 37ms/step - loss: 0.3850 - accuracy: 0.8255 - val_loss: 0.3335 - val_accuracy: 0.8570
Epoch 2/10
250/250 [==============================] - 9s 35ms/step - loss: 0.2527 - accuracy: 0.8975 - val_loss: 0.3377 - val_accuracy: 0.8624
Epoch 3/10
250/250 [==============================] - 8s 33ms/step - loss: 0.1978 - accuracy: 0.9228 - val_loss: 0.3634 - val_accuracy: 0.8539
Epoch 4/10
250/250 [==============================] - 8s 33ms/step - loss: 0.1556 - accuracy: 0.9410 - val_loss: 0.4351 - val_accuracy: 0.8512
Epoch 5/10
250/250 [==============================] - 8s 33ms/step - loss: 0.1188 - accuracy: 0.9559 - val_loss: 0.5028 - val_accuracy: 0.8470
Epoch 6/10
250/250 [==============================] - 8s 33ms/step - loss: 0.1041 - accuracy: 0.9634 - val_loss: 0.5142 - val_accuracy: 0.8447
Epoch 7/10
250/250 [==============================] - 8s 34ms/step - loss: 0.0795 - accuracy: 0.9730 - val_loss: 0.5249 - val_accuracy: 0.8380

In [ ]:
y_pred1=model1.predict_classes(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred1)


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[3429,  614],
       [ 660, 3297]])

In [ ]:

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.84075

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.84      0.85      0.84      4043
           1       0.84      0.83      0.84      3957

    accuracy                           0.84      8000
   macro avg       0.84      0.84      0.84      8000
weighted avg       0.84      0.84      0.84      8000

